In [1]:
!pip install praw
import praw

In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import math
import pandas as pd
import numpy as np

In [3]:
reddit = praw.Reddit(client_id="K3j6wiPOwUPduCsX3TuV7g",
                                client_secret="Gufs3iuqAhlJpZ9rOJfV-SEGruhpdQ",  
                                user_agent="WSB Scrapper CS410",
                                username="Academic_Jellyfish34",
                                password="123123AAA!")

In [4]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kcen4\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kcen4\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
list = ["AMD", "LCID", "PLTR", "RIVN"] 

def computeSentiment(ticker, subURL):
    
    sia = SIA()
    results = []
    comments = []
    aggComments = []
    
    try:
        page = reddit.submission(url=subURL)
        comments = page.comments
    except:
        return 0
    
    for comment in comments:
        try: 
            aggComments.append(comment.body)
        except:
            return 0
    
    for line in aggComments:
        scores = sia.polarity_scores(line)
        scores['line'] = line

        results.append(scores)
    
    df =pd.DataFrame.from_records(results)
    df['label'] = 0
    
    try:
        df.loc[df['compound'] > 0.1, 'label'] = 1
        df.loc[df['compound'] < -0.1, 'label'] = -1
    except:
        return 0
    
    averageScore = 0
    index = 0
    
    while index < len(df.label)-1:
        averageScore = averageScore + df.label[index]
        index += 1
    averageScore = averageScore/len(df.label) 
    
    return(averageScore)

output = []
data = {}

for stock in list:
    for submission in reddit.subreddit('wallstreetbets').search(stock, limit=10):
        data = {}
        data['Ticker'] = stock
        data['Avg Sentiment'] = computeSentiment(stock, submission.url)
        if data['Avg Sentiment'] == 0:
            continue
        output.append(data)
    
df = pd.DataFrame(output)

df.to_csv('WSB_Sentiment.csv', index=False) 